Copyright © 2017-2021 ABBYY Production LLC

In [1]:
#@title
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gradient bossting tutorial

This tutorial contains the following steps:

* Download dataset
* Compare different boosting builders

## Download dataset

*Note*: This section doesn't have any NeoML-specific code. It just downloads dataset from the internet. If you are not running this notebook, you may skip this section.

In this tutorial we will use 20newsgroups vectorized dataset from scikit-learn

In [2]:
from sklearn.datasets import fetch_20newsgroups_vectorized

train_data = fetch_20newsgroups_vectorized(subset='train')
test_data = fetch_20newsgroups_vectorized(subset='test')

## Compare different boosting builders

In order to compare different boosting builder types we need to:

* Prepare error function
* Fix other boosting parameters

In [3]:
import numpy as np

def accuracy(model, X, y):
    """Returns the accuracy of model on the given data"""
    correct = sum(1 for label, probs in zip(y, model.classify(X))
                  if label == np.argmax(probs))
    return float(correct)/len(y)


# These arguments will be used for every builder_type
shared_kwargs = {
    'loss' : 'binomial',
    'iteration_count' : 100,
    'learning_rate' : 0.1,
    'subsample' : 1.,
    'subfeature' : 0.25,
    'random_seed' : 1234,
    'max_depth' : 6,
    'max_node_count' : -1,
    'l1_reg' : 0.,
    'l2_reg' : 1.,
    'prune' : 0.,
    'thread_count' : 1,
}

Now we'll compare training speed and accuracy of different gradient boosting builders.

NeoML has several builder of gradient boostings:

* `full` - classic algorithm. If dataset is multi-class it uses one-versus-all approach
* `hist` - uses histograms of feature values when deciding to split node
* `multi_full` - classic with one modification: it always builds one ensemble, where each regressor performs regression with multiple values.

In [4]:
import time
import neoml

# Train and test boosting for every builder type
for builder in ['full', 'hist', 'multi_full']:
    start = time.time()
    boost_kwargs = { **shared_kwargs, 'builder_type' : builder}
    classifier = neoml.GradientBoost.GradientBoostClassifier(**boost_kwargs)
    model = classifier.train(train_data.data, train_data.target)
    run_time = time.time() - start
    acc = accuracy(model, test_data.data, test_data.target)
    print(f'{builder}  Accuracy: {acc:.4f}  Time: {run_time:.2f} sec.')

full  Accuracy: 0.7868  Time: 111.54 sec.
hist  Accuracy: 0.7926  Time: 198.95 sec.
multi_full  Accuracy: 0.6609  Time: 209.08 sec.
